In [1]:
# Importowanie bibliotek
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from random import shuffle
from matplotlib import cm

%matplotlib inline

# Zapewnienie powtarzalnośći losowanych wartości
np.random.seed(157)

# Wczytanie ramki danych
dfu = pd.read_csv('tae.data', header=None)

# Przypisanie etykiet
columns = ['English speaker', 'Course', 'Course instructor', 'Summer or regular semester', 'Class size', 'Class attribute']
dfu.columns = columns

# Znormalizowanie danych
df=dfu
df=((df-df.min())/(df.max()-df.min()))
df["Class attribute"]=dfu["Class attribute"]

In [2]:
# Stworzenie losowej próbki danych do zbioru testowego
valid = df.sample(15)
train = df.drop(valid.index)

# Podzielenie danych na wejściowe i wyjściowe
x = train.drop('Class attribute', axis=1).values
labels = train['Class attribute']
y = pd.get_dummies(train['Class attribute']).values

# Podzielenie danych na treningowe i testowe
xt = valid.drop('Class attribute', axis=1).values
labels = valid['Class attribute']
yt = pd.get_dummies(valid['Class attribute']).values

(136, 5)

In [3]:
def layer_sizes(X, Y):
    # Romiar warstwy wejściowej
    n_x = X.shape[1]
    # Rozmiar warstwy wyjściowej
    n_y = Y.shape[1]
    return (n_x, n_y)
        
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [4]:
def initialize_parameters(n_x, n_h1, n_h2, n_y):
    # Ustawienie losowych wartosci wag i ustawienie zerowych wartości progów
    W1 = np.random.randn(n_x,n_h1)*0.01
    b1 = np.zeros(n_h1)

    W2 = np.random.randn(n_h1,n_h2)*0.01
    b2 = np.zeros(n_h2)

    W3 = np.random.randn(n_h2,n_y)*0.01
    b3 = np.zeros(n_y)

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [5]:
def forward_propagation(X, parameters):
    # Pobranie wartości wag i progów ze słownika "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    # Warstwa 1
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)

    # Warstwa 2
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    
    # Warstwa 3
    Z3 = np.dot(A2, W3) + b3
    A3 = sigmoid(Z3)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2,
             "Z3": Z3,
             "A3": A3}
    
    return A3, cache

In [6]:
def compute_cost(A3, Y, parameters):
    m = Y.shape[0] # Ilość przykładów
    # cross entropy
    logprobs = np.multiply(np.log(A3), Y) + np.multiply((1 - Y), np.log(1 - A3))
    cost = - np.sum(logprobs) / m
    
    cost = np.squeeze(cost)
    
    return cost*100

In [7]:
def accuracy(output_vec, test_vec):
    equals = np.equal(np.argmax(test_vec, axis=1), np.argmax(output_vec, axis=1))
    acc = np.mean(equals)
    return acc

In [8]:
def adapt_learning_rate(learning_rate, xi_d, xi_i, er_r, cost, prev_cost):
    if cost > er_r*prev_cost:
        learning_rate*=xi_d
    elif cost < prev_cost:
        learning_rate*=xi_i
    prev_cost = cost
    return learning_rate, prev_cost

In [9]:
def backward_propagation(parameters, cache, X, Y):
    # Pobranie W1 - W3 ze słownika "parameters".
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
        
    # Pobranie A1-A3 ze słownika "cache".
    A1 = cache['A1']
    A2 = cache['A2']
    A3 = cache['A3']
    
    Z1 = cache['Z1']
    Z2 = cache['Z2']
    
    ########## Warstwa 3
    delta3 = A3-Y
    dW3 = np.dot(A2.T, delta3)
    db3 = delta3
    
    ########## Warstwa 2
    delta2 = np.dot(delta3, W3.T) * sigmoid_prime(Z2)
    dW2 = np.dot(A1.T, delta2)
    db2 = delta2
    
    ########## Warstwa 1
    delta1 = np.dot(delta2, W2.T) * sigmoid_prime(Z1)
    dW1 = np.dot(X.T, delta1)
    db1 = delta1

    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2,       
             "dW3": dW3,
             "db3": db3}
    return grads

In [10]:
def update_parameters(parameters, grads, learning_rate):
    # Pobranie wag i progów ze słownika "parameters".
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    # Pobranie gradientu ze słownika "grads".
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    dW3 = grads['dW3']
    db3 = grads['db3']

    # Aktualizacja wag i progów.
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1.sum(axis=0)

    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2.sum(axis=0)
    
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3.sum(axis=0)
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [11]:
def nn_model(X, Y, n_h1, n_h2, er_r, xi_i, xi_d, num_iterations, learning_rate):
    prev_cost = 1000.0
    n_x, n_y = layer_sizes(X, Y)
    
    parameters = initialize_parameters(n_x, n_h1, n_h2, n_y)
 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    for epoch in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A3, cache = forward_propagation(X, parameters)

        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A3, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)

        # CONSTANT LEARNING RATE:
        # learning_rate = 0.01
        
        # DECREMENTAL LEARNING RATE:
        # learning_rate *= 1/(1 + learning_rate/(epoch+1) * epoch)
        
        # ADAPTIVE LEARNING RATE:
        learning_rate, prev_cost = adapt_learning_rate(learning_rate, xi_d, xi_i, er_r, cost, prev_cost)

        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate)


    return parameters, A3

In [12]:
n_h1 = 68
n_h2 = 43
er_r = 1.016
xi_i = 1.12
xi_d = 0.7
parameters, A3 = nn_model(x, y, n_h1, n_h2, er_r, xi_i, xi_d, num_iterations = 4800, learning_rate = 0.01)
acc = accuracy(output_vec = A3, test_vec = y)
A3, cache = forward_propagation(xt, parameters)
acc_t = accuracy(output_vec = A3, test_vec = yt)
print("{}, {}, {}, {}".format(n_h1, n_h2, acc, acc_t))

(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)


(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)


(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)


(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)


(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
(5, 68)
